In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-23 20:58:31.524778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 20:58:31.629870: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-23 20:58:32.163798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



## Generate the Train/Dev/Test

In [3]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)
loader.sample(0.2)
loader.splitData()

source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

#data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['UberDuck'])

In [4]:
train_count = data_df[data_df['type'] == 'train'].shape[0]
dev_count = data_df[data_df['type'] == 'dev'].shape[0]
test_count = data_df[data_df['type'] == 'test'].shape[0]

print(f'# of Train instances: {train_count}')
print(f'# of Dev instances: {dev_count}')
print(f'# of Test instances: {test_count}')

# of Train instances: 3132
# of Dev instances: 1044
# of Test instances: 1046


In [5]:
data_df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ036-0097,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ037-0092,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ049-0102,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ042-0247,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ022-0036,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


## Generate the Features

In [6]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()

[NeMo I 2023-04-11 19:15:18 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-04-11 19:15:18 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-04-11 19:15:18 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-04-11 19:15:18 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-04-11 19:15:18 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-04-11 19:15:18 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-04-11 19:15:18 features:267] PADDING: 16
[NeMo I 2023-04-11 19:15:19 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:00<00:00,  3.60it/s]


In [7]:
cadence_manager = CadenceModelManager(data_df)
cad_feature_df, cad_feature_cols =  cadence_manager.run_cadence_feature_extraction_pipeline()
# TO DO: save the features down to CSV so can be loaded with features loader 

Normalizing amplitudes
Truncating silences
Truncating audio 100/5222 (2%)
Truncating audio 200/5222 (4%)
Truncating audio 300/5222 (6%)
Truncating audio 400/5222 (8%)
Truncating audio 500/5222 (10%)
Truncating audio 600/5222 (11%)
Truncating audio 700/5222 (13%)
Truncating audio 800/5222 (15%)
Truncating audio 900/5222 (17%)
Truncating audio 1000/5222 (19%)
Truncating audio 1100/5222 (21%)
Truncating audio 1200/5222 (23%)
Truncating audio 1300/5222 (25%)
Truncating audio 1400/5222 (27%)
Truncating audio 1500/5222 (29%)
Truncating audio 1600/5222 (31%)
Truncating audio 1700/5222 (33%)
Truncating audio 1800/5222 (34%)
Truncating audio 1900/5222 (36%)
Truncating audio 2000/5222 (38%)
Truncating audio 2100/5222 (40%)
Truncating audio 2200/5222 (42%)
Truncating audio 2300/5222 (44%)
Truncating audio 2400/5222 (46%)
Truncating audio 2500/5222 (48%)
Truncating audio 2600/5222 (50%)
Truncating audio 2700/5222 (52%)
Truncating audio 2800/5222 (54%)
Truncating audio 2900/5222 (56%)
Truncating au

[NeMo W 2023-04-11 19:18:10 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
      ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
    
[NeMo W 2023-04-11 19:18:10 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
      arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
    
[NeMo W 2023-04-11 19:18:10 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
      ret = ret.dtype.type(ret / rcount)
    
[NeMo W 2023-04-11 19:18:10 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
      return _methods._mean(a, axis=axis, dtype=dtype,
    
[NeMo W 2023-04-11 19:18:10 nemo_logging:349] /home/u

Extracting amplitude features
Creating dataframe
Complete


In [8]:
from packages.SavedFeatureLoader import loadFeatures
loadFeatures(data_df, 'openSmile', feature_filepath_col='file')

100%|██████████| 2/2 [00:23<00:00, 11.94s/it]


,type,id,architecture,path,label,multiclass_label,file,duration(seconds),audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,train,LJ036-0097,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,1.516,2.194382,0.138889,...,8.314812,0.705433,2.417853,2.364823,17.657010,0.431356,120.784520,59.592860,119.876070,43.886913
1,train,LJ037-0092,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,7.867,2.072287,0.521181,...,7.615157,0.588162,2.373873,2.376749,-19.999676,0.591689,105.293620,41.500652,112.774940,50.889038
2,train,LJ049-0102,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,10.061,2.531474,0.022022,...,9.435542,0.521164,2.820825,2.816656,19.999031,0.468180,125.002990,60.868553,121.830505,58.540085
3,train,LJ042-0247,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,8.598,2.106516,0.031690,...,10.765567,0.587056,3.045418,3.042028,19.999804,0.482918,125.950325,66.208720,126.476265,70.475105
4,train,LJ022-0036,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,5.614,2.439403,0.342960,...,7.804111,0.515232,2.693026,2.698434,-19.995700,0.604178,128.390840,60.624280,128.163800,60.512302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,test,LJ031-0196,UberDuck,/home/ubuntu/data/wavefake_data/generated_audi...,1,1,/home/ubuntu/data/wavefake_data/generated_audi...,5.712,1.764131,0.421986,...,8.104252,0.596381,2.855869,2.873493,-19.790556,0.453156,110.277570,56.519310,105.504060,51.461037
5218,test,LJ032-0007,UberDuck,/home/ubuntu/data/wavefake_data/generated_audi...,1,1,/home/ubuntu/data/wavefake_data/generated_audi...,7.279,2.077179,0.069444,...,7.791924,0.551224,2.659146,2.664824,-19.994352,0.576932,121.886660,57.850100,102.889930,54.073185
5219,test,LJ006-0169,UberDuck,/home/ubuntu/data/wavefake_data/generated_audi...,1,1,/home/ubuntu/data/wavefake_data/generated_audi...,2.670,1.703546,0.092308,...,3.614227,0.322753,2.426772,2.447754,-19.531850,0.691352,108.521355,45.281880,113.448450,63.423935
5220,test,LJ021-0140,UberDuck,/home/ubuntu/data/wavefake_data/generated_audi...,1,1,/home/ubuntu/data/wavefake_data/generated_audi...,6.304,1.579996,0.012841,...,6.578919,0.452569,2.409700,2.409105,20.000000,0.631341,92.980340,46.002120,88.994415,44.772747


## Predict and Evaluate

In [8]:
em_model_manager_bin = ModelManager('logreg', em_feature_df, em_feature_cols, merge_train_dev=True)
em_model_manager_bin.trainPredict(label_col='label')

[NeMo W 2023-04-11 19:21:55 nemo_logging:349] /home/ubuntu/MultiModalDeepFake/packages/ModelManager.py:61: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
      self.X_train = self.train[self.feature_cols].copy()
    
[NeMo W 2023-04-11 19:21:55 nemo_logging:349] /home/ubuntu/MultiModalDeepFake/packages/ModelManager.py:68: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
      self.X_test = self.test[self.feature_cols].copy()
    


(0.9990439770554493, 0.20646261802843452)

In [9]:
em_model_manager_multi = ModelManager('logreg', em_feature_df, em_feature_cols, merge_train_dev=True)
em_model_manager_multi.trainPredict(label_col='multiclass_label')

[NeMo W 2023-04-11 19:21:55 nemo_logging:349] /home/ubuntu/MultiModalDeepFake/packages/ModelManager.py:61: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
      self.X_train = self.train[self.feature_cols].copy()
    
[NeMo W 2023-04-11 19:21:55 nemo_logging:349] /home/ubuntu/MultiModalDeepFake/packages/ModelManager.py:68: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
      self.X_test = self.test[self.feature_cols].copy()
    


(0.9990439770554493, 0.20646261802843452)

In [10]:
cad_model_manager_bin = ModelManager('logreg', cad_feature_df, cad_feature_cols, merge_train_dev=True)
cad_model_manager_bin.trainPredict(label_col='label')

(0.570334928229665, 0.6734536823903307)

In [11]:
cad_model_manager_multi = ModelManager('logreg', cad_feature_df, cad_feature_cols, merge_train_dev=True)
cad_model_manager_multi.trainPredict(label_col='multiclass_label')

(0.570334928229665, 0.6734536823903307)

In [12]:
print('Binary Performance')
print('---------------------------')
print(f'Embeddings Accuracy: {em_model_manager_bin.accuracy}')
print(f'Cadence Accuracy: {cad_model_manager_bin.accuracy}')
print(f'Embeddings Loss: {em_model_manager_bin.log_loss_value}')
print(f'Cadence Loss: {cad_model_manager_bin.log_loss_value}')

Binary Performance
---------------------------
Embeddings Accuracy: 0.9990439770554493
Cadence Accuracy: 0.570334928229665
Embeddings Accuracy: 0.20646261802843452
Cadence Accuracy: 0.6734536823903307


In [13]:
am.predCorrelation((em_model_manager_bin, cad_model_manager_bin), label_type='binary')

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1046 and the array at index 1 has size 1045

In [14]:
print('Multi-Class Performance')
print('---------------------------')
print(f'Embeddings Accuracy: {em_model_manager_multi.accuracy}')
print(f'Cadence Accuracy: {cad_model_manager_multi.accuracy}')
print(f'Embeddings Loss: {em_model_manager_multi.log_loss_value}')
print(f'Cadence Loss: {cad_model_manager_multi.log_loss_value}')

Multi-Class Performance
---------------------------
Embeddings Accuracy: 0.9990439770554493
Cadence Accuracy: 0.570334928229665
Embeddings Accuracy: 0.20646261802843452
Cadence Accuracy: 0.6734536823903307


In [ ]:
am.predCorrelation((em_model_manager_multi, cad_model_manager_multi), label_type='multiclass')